# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成計數編碼以及搭配邏輯斯迴歸的預測 (In[4], Out[4], In[5], Out[5]) 
- 仿造範例, 完成雜湊編碼, 以及計數編碼+雜湊編碼 搭配邏輯斯迴歸的預測 (In[6], Out[6], In[7], Out[7]) 
- 試著回答上述執行結果的觀察

# 作業1
* 參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 目標均值編碼三種轉換後，  
與其他類別型欄位一起預估生存機率

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

# 讀取訓練與測試資料
# !wget http://ai100.cupoy.com/file-download/part02/titanic_test.csv
# !wget http://ai100.cupoy.com/file-download/part02/titanic_train.csv

data_path = './'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


In [3]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
# penalties = ['l1', 'l2', 'elasticnet', 'none']
# Logistic Regression supports only solvers in ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'], got elasticnet.

start_cp = time.time()
df_temp = pd.DataFrame()

for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])

estimator = LogisticRegression(solver='newton-cg')
train_X = df_temp[:train_num]

score_cp = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
time_cp = time.time() - start_cp

print(f'score_cp : {score_cp}')
print(f'time_cp: {time_cp} sec')

df_temp.head()

score_cp : 0.7789000729487724
time_cp: 0.29859352111816406 sec


,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [4]:
# 加上 'Cabin' 欄位的計數編碼
count_df = df.groupby(['Cabin'])['Ticket'].agg({'Cabin_Count':'size'}).reset_index()
df_Cabin = pd.merge(df, count_df, on=['Cabin'], how='left')
# count_df.sort_values(by=['Cabin_Count'], ascending=False).head(10)


# 'Cabin'計數編碼 + 邏輯斯迴歸
start1 = time.time()
df_temp1 = pd.DataFrame()

for c in df.columns:
    df_temp1[c] = LabelEncoder().fit_transform(df[c])

estimator = LogisticRegression(solver='newton-cg')

df_temp1['Cabin_Count'] = count_df['Cabin_Count']
df_temp1 = df_temp1.fillna(0)
train_X = df_temp1[:train_num]

score1 = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
time1 = time.time() - start1
print(f'score1 : {score1}')
print(f'time1 : {time1} sec')
df_temp1.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """Entry point for launching an IPython kernel.


score1 : 0.7621781006863284
time1 : 0.34235477447509766 sec


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count
0,155,1,720,185,3,1.0
1,286,0,816,106,0,1.0
2,523,0,914,185,3,1.0
3,422,0,65,70,3,1.0
4,22,1,649,185,3,1.0


In [5]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
start2 = time.time()
df_temp2 = pd.DataFrame()

df_temp2['Cabint_Hash'] = df['Cabin'].map(lambda x:hash(x) % 10)

for c in df.columns:
    df_temp2[c] = LabelEncoder().fit_transform(df[c])

estimator = LogisticRegression(solver='newton-cg')
train_X = df_temp2[:train_num]

score2 = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
time2 = time.time() - start2
print(f'score2 : {score2}')
print(f'time2 : {time2} sec')

df_temp2.head()

score2 : 0.7766528110101933
time2 : 0.3129384517669678 sec


,Cabint_Hash,Name,Sex,Ticket,Cabin,Embarked
0,5,155,1,720,185,3
1,2,286,0,816,106,0
2,5,523,0,914,185,3
3,7,422,0,65,70,3
4,5,22,1,649,185,3


In [6]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
start3 = time.time()
df_temp3 = pd.DataFrame()

df_temp3['Cabint_Hash'] = df['Cabin'].map(lambda x:hash(x) % 10)

for c in df.columns:
    df_temp3[c] = LabelEncoder().fit_transform(df[c])

estimator = LogisticRegression(solver='newton-cg')
df_temp3['Cabin_Count'] = count_df['Cabin_Count']
df_temp3 = df_temp3.fillna(0)
train_X = df_temp3[:train_num]

score3 = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
time3 = time.time() - start3
print(f'score3 : {score3}')
print(f'time3 : {time3} sec')
df_temp3.head()

score3 : 0.7576835768091703
time3 : 0.3427863121032715 sec


,Cabint_Hash,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count
0,5,155,1,720,185,3,1.0
1,2,286,0,816,106,0,1.0
2,5,523,0,914,185,3,1.0
3,7,422,0,65,70,3,1.0
4,5,22,1,649,185,3,1.0


# 作業2
* 承上題，三者比較效果何者最好?

In [7]:
df_score = {"score":[score_cp,score1,score2,score3],"time":[time_cp,time1,time2,time3]}
df_score = pd.DataFrame(df_score,index=["標籤編碼 + 邏輯斯迴歸","'Cabin'計數編碼 + 邏輯斯迴歸","'Cabin'特徵雜湊 + 邏輯斯迴歸","'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸"])
df_score

,score,time
標籤編碼 + 邏輯斯迴歸,0.778900,0.298594
'Cabin'計數編碼 + 邏輯斯迴歸,0.762178,0.342355
'Cabin'特徵雜湊 + 邏輯斯迴歸,0.776653,0.312938
'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸,0.757684,0.342786
